<a href="https://colab.research.google.com/github/ljs7463/AnalysisProject/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98/analytics/xgb_ligthgbm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# 시각화 폰트 설정
if os.name =='posix':
    plt.rc("font", family = "AppleGothic")

else:
    plt.rc("font", family = "Malgun Gothic")

# 경고문자 무시
warnings.filterwarnings(action='ignore')

In [3]:
# 쥬피터 실행시
# train_path = '../data/open/train.csv'
# test_path = '../data/open/test.csv'
# info_path = '../data/open/data_info.csv'
# sub_path = '../data/open/sample_submission.csv'
# df_train = pd.read_csv(train_path)
# df_test = pd.read_csv(test_path)
# df_info = pd.read_csv(info_path)
# df_sub = pd.read_csv(sub_path)

# 코랩 실행시
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')
df_info = pd.read_csv('data_info.csv')

In [4]:
## Label Encoding

## train데이터
df_train['preferred_difficulty_level'] = pd.factorize(df_train['preferred_difficulty_level'])[0]
df_train['subscription_type'] = pd.factorize(df_train['subscription_type'])[0]

## test데이터
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_train[i] = scaler.fit_transform(df_train[i])
scaler = StandardScaler()
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_test[i] = scaler.fit_transform(df_test[i])

In [6]:
# Delete user_id
df_train = df_train.drop(columns = 'user_id')

# split target
x = df_train[list(df_train.columns[:-1])]
y = df_train['target']

In [7]:
# Delete user_id
df_test = df_test.drop(columns = 'user_id')

# split target
new_x = df_test


In [8]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
import pandas as pd

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

param_grid = {
    'max_depth': [3,4,5,6],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100,200,300,400,500, 600],
    'colsample_bytree': [0.3, 0.7, 0.9]
}

# XGBoost 분류기 초기화
xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Macro F1 스코어를 사용하기 위한 스코어러 생성
macro_f1_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring = macro_f1_scorer, n_jobs=-1, verbose=1)

# GridSearchCV 실행
grid_search.fit(x_train, y_train)

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 정확도:", grid_search.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
최적의 파라미터: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 600}
최고 평균 정확도: 0.4933807068078523


In [9]:
# 최적의 모델 가져오기
best_model = grid_search.best_estimator_
y_pred = best_model.predict_proba(x_test)[:,1]

## 정밀도와 재현율 계산
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

# 정밀도와 재현율의 최적 임계값 찾기
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))


Best Threshold=0.120191, F-Score=0.767


In [10]:
# 임계값 적용을 시키기
from sklearn.preprocessing import Binarizer
new_threshold = thresholds[ix]

prediction = best_model.predict_proba(new_x)
# 새로운 임계값적용
prediction_proba =prediction[:,1].reshape(-1,1)
binarizer =Binarizer(threshold = new_threshold).fit(prediction_proba)
final_prediction = binarizer.transform(prediction_proba)

In [13]:
pd.DataFrame(final_prediction).value_counts()

1.0    9972
0.0      28
dtype: int64

# light gbm

In [16]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
param_grid = {
    'max_depth': [5,7 ,9, 11],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [ 600,700,800,900,1000],
    'colsample_bytree': [0.3, 0.7, 0.9],
    'num_leaves': [31, 50, 70],
}
# LightGBM 분류기 초기화
lgb_classifier = lgb.LGBMClassifier()

# Macro F1 스코어를 사용하기 위한 스코어러 생성
macro_f1_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV 설정s
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid, cv=3, scoring=macro_f1_scorer, n_jobs=-1, verbose=1)

# GridSearchCV 실행
grid_search.fit(x_train, y_train)

# 결과 출력
print("최적의 파라미터:", grid_search.best_params_)
print("최고 평균 Macro F1 점수:", grid_search.best_score_)

# 최적의 모델 가져오기
best_model = grid_search.best_estimator_

# 예측
y_pred = best_model.predict(x_test)

# Macro F1 점수 평가
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Test Set Macro F1 Score: {macro_f1}")

Fitting 3 folds for each of 540 candidates, totalling 1620 fits
[LightGBM] [Info] Number of positive: 4957, number of negative: 3043
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000620 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 916
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.619625 -> initscore=0.487957
[LightGBM] [Info] Start training from score 0.487957
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM